In [2]:
%load_ext autoreload
%autoreload 2
import hyperspy.api as hs
%matplotlib qt4
import matplotlib.pyplot as plt
import numpy as np
from hyperspy.zeta import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Find zeta factor

In [77]:
thickness_map=hs.load('../Data/unheated/Thickness Map.dm3')
thickness_map.plot()
l = 104 # From Andreas
tuh = thickness_map.isig[100.:200.,70.:190.].data.mean()*l

In [80]:
thickness_map_heated=hs.load('../Data/160506_SC239_X/Thickness Map.dm3')
thickness_map_heated.plot()
l=104
th=thickness_map_heated.isig[86.:185.,71.:151.].data.mean()*l
print(th,tuh)

74.6921830177 121.975335121


In [79]:
%load_ext autoreload
%autoreload 2

plt.close('all')
IW = 1.2
tilt = -np.sqrt(2.3**2+.6**2)

p_GaAs = 5317.6
p_Au = hs.material.elements.Au.Physical_properties.density_gcm3*1000
p_Ge = hs.material.elements.Ge.Physical_properties.density_gcm3*1000
p_Pd = hs.material.elements.Pd.Physical_properties.density_gcm3*1000
 
s_A = hs.load('../Data/unheated/A_EDS GaAs.hdf5')
s_Cheat = hs.load('../Data/160506_SC239_X/C_EDS Spectrum Image GaAs.hdf5')
s_A1 = hs.load('../Data/unheated/A_EDS Ge.hdf5')
s_A2 = hs.load('../Data/unheated/A_EDS Pd.hdf5')
s_C = hs.load('../Data/unheated/C_EDS Spectrum Image.dm3')
s_D = hs.load('../Data/unheated/D_EDS Spectrum Image.dm3')
s_E = hs.load('../Data/unheated/E_EDS Spectrum Image.dm3')
s_B1=hs.load('../Data/unheated/B_EDS_1.hdf5')
s_B2=hs.load('../Data/unheated/B_EDS_2.hdf5')
s_B3=hs.load('../Data/unheated/B_EDS_3.hdf5')

s_GaAs=[s_A,s_Cheat]
s_Ge=[s_A2,s_D,s_B2]
s_Pd=[s_A1, s_E,s_B1]
s_Au=[s_C,s_B3]
names=[['A','Cheat'],['A','D','B'],['A','E','B'],['C','B']]
curA=(.1769+.1747)/2
curB=.1747
probe_current = [[curA,(.1194+.1181)/2],[curA,(.1747+.1750)/2,curB],[curA,.1750,curB],[.1747,curB]]
elements = [['As','Ga'],['Ge'],['Pd'],['Au']]
s = [s_GaAs,s_Ge,s_Pd,s_Au]
time = [[.01,.1],[.01,.1,.01],[.01,.1,.01],[.1,.01]]
density_list=[p_GaAs,p_Ge,p_Pd,p_Au]

thickness_list=[[tuh,th],[tuh,tuh,tuh],[tuh,tuh,tuh],[tuh,tuh]]
BW_C = [9.25, 9.45,10.1,10.3]
BW_D = [9.4,9.6,10.2,10.4]
BW_E = [2.2,2.4,3.7,3.9]
BW_C =[1.85,1.91,2.53,2.63] #gold M-line
BW_D = [.6,.7,1.4,1.5] #Ge L-line

GeL=[.6,.7,1.4,1.5]
AuM=[1.88,1.96,3.7,3.9]
AuL=[7.3,7.5,10.6,10.8]
GaK=[ 9.79,9.99, 11.09, 11.29]
AsK=[ 8.47,8.66,9.67,9.86]
GaAsL=[.6,.85,1.5,1.6]
GeK=[9.58,9.63,10.3,10.4]
PdL= [2.2,2.4,3.7,3.9]


# bw_list=[BW_A,BW_A,BW_C,BW_D,BW_E,BW_E,BW_D,BW_C]
bw_list=[[[GaK,AsK],[GaAsL,GaAsL]],[[GeK],[GeL]],[[PdL]],[[AuL],[AuM]]]

COMP_GA_AS_WT=(hs.material.atomic_to_weight([.5,.5],['As','Ga'])/100)
comp_list = [COMP_GA_AS_WT,[1],[1],[1]]

lines=[[['As_Ka','Ga_Ka'],['As_La','Ga_La']],[['Ge_Ka'],['Ge_La']],[['Pd_La']],[['Au_La'],['Au_Ma']]]
plt.close('all')
hfile=open('../zetafactors-thickness-corr-GaAs.txt','w+')
plt.ion()
for edxname,edxlist, currentlist, density, element, bwlist, comp, tlist, linelist, thicknesslist in zip(names,s,probe_current, density_list, elements,bw_list,comp_list, time, lines,thickness_list):
    for name,edx, current, t,thickness in zip(edxname,edxlist,currentlist, tlist,thicknesslist):
        if 'Ga' not in element:
            continue
        time = edx.data.shape[0]*edx.data.shape[1]*t
        edx=edx.sum('x').sum('y')
        edx.set_elements(element)
        print(element,'\n --------------')
        i = 0
        for bw, line in zip(bwlist,linelist):
            edx=edx.copy()
            i += 1
            edx.metadata.General.title= name + ' ' +str(line) +' '+str(i)
            edx.set_lines(line)
            edx.plot(background_windows=bw)
            edx.set_microscope_parameters(beam_current=current, tilt_stage=tilt, real_time=time, elevation_angle=24.3, azimuth_angle=0.0)
#             if len(line)==1:
#                 bw=bw[0]
            print(bw,line)
            s_int=edx.get_lines_intensity(background_windows=bw, integration_windows=IW,plot_result=0, only_lines=line)
            zeta = determine_zeta_factor(edx, s_int, comp, thickness, density)
           
            print(edx.metadata.General.title)
            for i in range(len(zeta)):
                print('zeta',np.nanmean(zeta[i].data.flatten()))            
                hfile.write(str(edx.metadata.General.title)+' '+str(np.nanmean(zeta[i].data.flatten()))+'\n')
    #             print('std',np.std(z.data))
    
#         break
#     break
hfile.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['As', 'Ga'] 
 --------------
[[9.79, 9.99, 11.09, 11.29], [8.47, 8.66, 9.67, 9.86]] ['As_Ka', 'Ga_Ka']
A ['As_Ka', 'Ga_Ka'] 1
zeta 706.458581674
zeta 607.860488895
[[0.6, 0.85, 1.5, 1.6], [0.6, 0.85, 1.5, 1.6]] ['As_La', 'Ga_La']
A ['As_La', 'Ga_La'] 2
zeta 954.913543881
zeta 825.715725567
['As', 'Ga'] 
 --------------
[[9.79, 9.99, 11.09, 11.29], [8.47, 8.66, 9.67, 9.86]] ['As_Ka', 'Ga_Ka']
Cheat ['As_Ka', 'Ga_Ka'] 1
zeta 575.645138935
zeta 486.275256999
[[0.6, 0.85, 1.5, 1.6], [0.6, 0.85, 1.5, 1.6]] ['As_La', 'Ga_La']
Cheat ['As_La', 'Ga_La'] 2
zeta 600.052264392
zeta 512.442033293


In [2]:
import matplotlib.pyplot as plt
plt.close('all')

In [51]:
edx.metadata.General.title

'123'